In [96]:
import warnings
warnings.filterwarnings('ignore')

# Information

This pipeline will generate and publish a CNN model ready to predict trained on CIFAR-10 Dataset. This notebook contains entire pipeline and data handling visualisation. It can be served in a notebook instance by Kubeflow mounting the tfx PersistentVolume. <br>

A file called cifar-10.tar.gz will be created. This is the pipeline file you should upload into Kubeflow UI. After the upload, the pipeline is ready to go! <br>

NEXT STEP: Running Kubeflow on Kubernetes Engine and Microsoft Azure: https://www.kubeflow.org/docs/distributions/azure/ 

# Import packages

In [97]:
# older version
# import os
# import tfx
# import tensorflow as tf
# import tensorflow_data_validation as tfdv
# import tensorflow_model_analysis as tfma
# import tempfile
# import absl
# from tfx.components.evaluator.component import Evaluator
# from tfx.components.example_gen.import_example_gen.component import ImportExampleGen
# from tfx.components.example_validator.component import ExampleValidator
# from tfx.components.model_validator.component import ModelValidator
# from tfx.components.pusher.component import Pusher
# from tfx.components.schema_gen.component import SchemaGen
# from tfx.components.statistics_gen.component import StatisticsGen
# from tfx.components.trainer.component import Trainer
# from tfx.components.transform.component import Transform
# from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
# from tfx.proto import example_gen_pb2
# from tfx.proto import evaluator_pb2
# from tfx.proto import pusher_pb2
# from tfx.proto import trainer_pb2
# # Fixing this below issue: https://colab.research.google.com/github/tensorflow/tfx/blob/master/docs/tutorials/tfx/components_keras.ipynb#scrollTo=ad5JLpKbf6sN
# # from tfx.utils.dsl_utils import external_input
# # from tfx.utils.dsl_utils import external_input
# from tfx.utils.types import get_split_uri

In [98]:
import os
import tfx
import tensorflow as tf
import tensorflow_data_validation as tfdv
import tensorflow_model_analysis as tfma
import tempfile
import absl

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import example_gen_pb2
from tfx.proto import evaluator_pb2
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.types.artifact_utils import get_split_uri

from tfx import v1 as tfx

import pprint
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

The tfx.orchestration.experimental.interactive.notebook_extensions.skip extension is already loaded. To reload it, use:
  %reload_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip


# Check library versions

In [99]:
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.7.0
TFX version: 1.5.0


# Set up pipeline paths

In [100]:
# tf.enable_eager_execution()

In [101]:
# # # This is the root directory for your TFX pip package installation.
# # _tfx_root = tfx.__path__[0]

# # # This is the directory containing the built-in TFX Cifar10 Pipeline example.
# # _taxi_root = os.path.join(_tfx_root, 'examples/cifar-10')

# Path to tfrecord directory
# _root = '.'
_root = os.getcwd()
_data_root = _root + '/data'

# This is the path where your model will be pushed for serving.
_serving_model_dir = os.path.join(tempfile.mkdtemp(),'serving_model/cifar-10')

# Set up logging.
absl.logging.set_verbosity(absl.logging.INFO)

In [102]:
context = InteractiveContext()

# TFX Pipeline Component

## ExampleGen

https://www.tensorflow.org/tfx/guide/examplegen <br>

The ExampleGen TFX Pipeline component ingests data into TFX pipelines. It consumes external files/services to generate Examples which will be read by other TFX components. It also provides consistent and configurable partition, and shuffles the dataset for ML best practice. <br>

- Consumes: Data from external data sources such as CSV, TFRecord, Avro, Parquet and BigQuery.
- Emits: tf.Example records, tf.SequenceExample records, or proto format, depending on the payload format.

In [103]:
# Older version
    
# input_split = example_gen_pb2.Input(splits=[
#     example_gen_pb2.Input.Split(name='train', pattern='train.tfrecord'),
#     example_gen_pb2.Input.Split(name='eval', pattern='test.tfrecord')
# ])

# examples = external_input(_data_root)
# example_gen = ImportExampleGen(input_base=examples, input_config=input_split)

# context.run(example_gen)

In [104]:
# Input train split is 'input_dir/train/*', eval split is 'input_dir/eval/*'.
# Output splits are generated one-to-one mapping from input splits.
input_split = example_gen_pb2.Input(splits=[
    example_gen_pb2.Input.Split(name='train', pattern='train.tfrecord'),
    example_gen_pb2.Input.Split(name='eval', pattern='test.tfrecord')
])

example_gen = tfx.components.ImportExampleGen(input_base=_data_root, input_config=input_split)

context.run(example_gen)

INFO:absl:Running driver for ImportExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for ImportExampleGen
INFO:absl:Generating examples.
INFO:absl:Reading input TFRecord data /Users/tommybillywijaya/Documents/_Main_Document/*Projects/JFC/cifar-10/data/train.tfrecord.
INFO:absl:Reading input TFRecord data /Users/tommybillywijaya/Documents/_Main_Document/*Projects/JFC/cifar-10/data/test.tfrecord.
INFO:absl:Examples generated.
INFO:absl:Running publisher for ImportExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ImportExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 14
        uri: "/var/folders/fm/1byvh8pj2hbf1yl82_qv2q0r0000gn/T/tfx-interactive-2022-01-24T13_46_15.376117-2wg_qh1h/ImportExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "file_format"
          value {
            string_value: "tfrecords_gzip"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:train,num_files:1,total_bytes:67847507,xor_checksum:1627476290,sum_checksum:1627476290\nsplit:eval,num_files:1,total_bytes:13573009,xor_checksum:1627476290,sum_checksum:1627476290"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        base_type: DATASET
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [105]:
example_gen.outputs

{'examples': Channel(
     type_name: Examples
     artifacts: [Artifact(artifact: id: 1
 type_id: 14
 uri: "/var/folders/fm/1byvh8pj2hbf1yl82_qv2q0r0000gn/T/tfx-interactive-2022-01-24T13_46_15.376117-2wg_qh1h/ImportExampleGen/examples/1"
 properties {
   key: "split_names"
   value {
     string_value: "[\"train\", \"eval\"]"
   }
 }
 custom_properties {
   key: "file_format"
   value {
     string_value: "tfrecords_gzip"
   }
 }
 custom_properties {
   key: "input_fingerprint"
   value {
     string_value: "split:train,num_files:1,total_bytes:67847507,xor_checksum:1627476290,sum_checksum:1627476290\nsplit:eval,num_files:1,total_bytes:13573009,xor_checksum:1627476290,sum_checksum:1627476290"
   }
 }
 custom_properties {
   key: "payload_format"
   value {
     string_value: "FORMAT_TF_EXAMPLE"
   }
 }
 custom_properties {
   key: "span"
   value {
     int_value: 0
   }
 }
 custom_properties {
   key: "state"
   value {
     string_value: "published"
   }
 }
 custom_properties {
   ke

Let's examine the output artifacts of ExampleGen. This component produces two artifacts, training examples and evaluation examples:

In [106]:
artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

["train", "eval"] /var/folders/fm/1byvh8pj2hbf1yl82_qv2q0r0000gn/T/tfx-interactive-2022-01-24T13_46_15.376117-2wg_qh1h/ImportExampleGen/examples/1


We can also take a look at the first three training examples:

In [11]:
# Get the URI of the output artifact representing the training examples, which is a directory
train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(3):
    serialized_example = tfrecord.numpy()
    example = tf.train.Example()
    example.ParseFromString(serialized_example)
    pp.pprint(example)

features {
  feature {
    key: "image_raw"
    value {
      bytes_list {
        value: "\377\330\377\340\000\020JFIF\000\001\001\001\001,\001,\000\000\377\333\000C\000\002\001\001\001\001\001\002\001\001\001\002\002\002\002\002\004\003\002\002\002\002\005\004\004\003\004\006\005\006\006\006\005\006\006\006\007\t\010\006\007\t\007\006\006\010\013\010\t\n\n\n\n\n\006\010\013\014\013\n\014\t\n\n\n\377\333\000C\001\002\002\002\002\002\002\005\003\003\005\n\007\006\007\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\377\300\000\021\010\000 \000 \003\001\"\000\002\021\001\003\021\001\377\304\000\037\000\000\001\005\001\001\001\001\001\001\000\000\000\000\000\000\000\000\001\002\003\004\005\006\007\010\t\n\013\377\304\000\265\020\000\002\001\003\003\002\004\003\005\005\004\004\000\000\001}\001\002\003\000\004\021\005\022!1A\006\023Qa\007\"q\0242\201\221\241\010#B\261\301\025R\321\360$3br\202\t\n\026\027\030\031\032%&\'()*456789:CDEFGHIJST

We can also take a look at the first three testing examples:

In [36]:
# Get the URI of the output artifact representing the training examples, which is a directory
eval_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-eval')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(eval_uri, name)
                      for name in os.listdir(eval_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(3):
    serialized_example = tfrecord.numpy()
    example = tf.train.Example()
    example.ParseFromString(serialized_example)
    pp.pprint(example)

features {
  feature {
    key: "image_raw"
    value {
      bytes_list {
        value: "\377\330\377\340\000\020JFIF\000\001\001\001\001,\001,\000\000\377\333\000C\000\002\001\001\001\001\001\002\001\001\001\002\002\002\002\002\004\003\002\002\002\002\005\004\004\003\004\006\005\006\006\006\005\006\006\006\007\t\010\006\007\t\007\006\006\010\013\010\t\n\n\n\n\n\006\010\013\014\013\n\014\t\n\n\n\377\333\000C\001\002\002\002\002\002\002\005\003\003\005\n\007\006\007\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\377\300\000\021\010\000 \000 \003\001\"\000\002\021\001\003\021\001\377\304\000\037\000\000\001\005\001\001\001\001\001\001\000\000\000\000\000\000\000\000\001\002\003\004\005\006\007\010\t\n\013\377\304\000\265\020\000\002\001\003\003\002\004\003\005\005\004\004\000\000\001}\001\002\003\000\004\021\005\022!1A\006\023Qa\007\"q\0242\201\221\241\010#B\261\301\025R\321\360$3br\202\t\n\026\027\030\031\032%&\'()*456789:CDEFGHIJST

Now that ExampleGen has finished ingesting the data, the next step is data analysis.

## StatisticsGen

https://www.tensorflow.org/tfx/guide/statsgen <br>

The StatisticsGen TFX pipeline component generates features statistics over both training and serving data, which can be used by other pipeline components. StatisticsGen uses Beam to scale to large datasets.

- Consumes: datasets created by an ExampleGen pipeline component.
- Emits: Dataset statistics.

In [107]:
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])

context.run(statistics_gen)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Generating statistics for split train.
INFO:absl:Statistics for split train written to /var/folders/fm/1byvh8pj2hbf1yl82_qv2q0r0000gn/T/tfx-interactive-2022-01-24T13_46_15.376117-2wg_qh1h/StatisticsGen/statistics/2/Split-train.
INFO:absl:Generating statistics for split eval.
INFO:absl:Statistics for split eval written to /var/folders/fm/1byvh8pj2hbf1yl82_qv2q0r0000gn/T/tfx-interactive-2022-01-24T13_46_15.376117-2wg_qh1h/StatisticsGen/statistics/2/Split-eval.
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 2
        type_id: 16
        uri: "/var/folders/fm/1byvh8pj2hbf1yl82_qv2q0r0000gn/T/tfx-interactive-2022-01-24T13_46_15.376117-2wg_qh1h/StatisticsGen/statistics/2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 16
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        base_type: STATISTICS
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

After StatisticsGen finishes running, we can visualize the outputted statistics. Try playing with the different plots!

In [108]:
#older version 
# artifact_list = statistics_gen.outputs['statistics'].get()
# train_artifact_uri = get_split_uri(artifact_list, 'train')
# train_stats_path = os.path.join(train_artifact_uri, 'stats_tfrecord')
# eval_artifact_uri = get_split_uri(artifact_list, 'eval')
# eval_stats_path = os.path.join(eval_artifact_uri, 'stats_tfrecord')

# train_stats = tfdv.load_statistics(train_stats_path)
# eval_stats = tfdv.load_statistics(eval_stats_path)

# tfdv.visualize_statistics(lhs_statistics=eval_stats, rhs_statistics=train_stats,
#                           lhs_name='EVAL_DATASET', rhs_name='TRAIN_DATASET')

In [109]:
context.show(statistics_gen.outputs['statistics'])

## SchemaGen

https://www.tensorflow.org/tfx/guide/schemagen

Some TFX components use a description of your input data called a schema. The schema is an instance of schema.proto. It can specify data types for feature values, whether a feature has to be present in all examples, allowed value ranges, and other properties. A SchemaGen pipeline component will automatically generate a schema by inferring types, categories, and ranges from the training data.

- Consumes: statistics from a StatisticsGen component
- Emits: Data schema proto

In [110]:
schema_gen = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=True)

context.run(schema_gen)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for SchemaGen
INFO:absl:Processing schema from statistics for split train.
INFO:absl:Processing schema from statistics for split eval.
INFO:absl:Schema written to /var/folders/fm/1byvh8pj2hbf1yl82_qv2q0r0000gn/T/tfx-interactive-2022-01-24T13_46_15.376117-2wg_qh1h/SchemaGen/schema/3/schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 18
        uri: "/var/folders/fm/1byvh8pj2hbf1yl82_qv2q0r0000gn/T/tfx-interactive-2022-01-24T13_46_15.376117-2wg_qh1h/SchemaGen/schema/3"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 18
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

SchemaGen will take as input the statistics that we generated with StatisticsGen, looking at the training split by default.

In [111]:
# older version
# schema_dir = infer_schema.outputs['schema'].get()[0].uri
# schema_path = os.path.join(schema_dir, 'schema.pbtxt')

# schema = tfdv.load_schema_text(schema_path)
# tfdv.display_schema(schema)

In [112]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'image_raw',BYTES,required,,-
'label',INT,required,,-


## ExampleValidator

https://www.tensorflow.org/tfx/guide/exampleval

The ExampleValidator pipeline component identifies anomalies in training and serving data. It can detect different classes of anomalies in the data. For example it can: perform validity checks by comparing data statistics against a schema that codifies expectations of the user, detect training-serving skew by comparing training and serving data, and detect data drift by looking at a series of data.

The ExampleValidator pipeline component identifies any anomalies in the example data by comparing data statistics computed by the StatisticsGen pipeline component against a schema. The inferred schema codifies properties which the input data is expected to satisfy, and can be modified by the developer.

- Consumes: A schema from a SchemaGen component, and statistics from a StatisticsGen component.
- Emits: Validation results

In [113]:
example_validator = tfx.components.ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])

context.run(example_validator)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for ExampleValidator
INFO:absl:Validating schema against the computed statistics for split train.
INFO:absl:Validation complete for split train. Anomalies written to /var/folders/fm/1byvh8pj2hbf1yl82_qv2q0r0000gn/T/tfx-interactive-2022-01-24T13_46_15.376117-2wg_qh1h/ExampleValidator/anomalies/4/Split-train.
INFO:absl:Validating schema against the computed statistics for split eval.
INFO:absl:Validation complete for split eval. Anomalies written to /var/folders/fm/1byvh8pj2hbf1yl82_qv2q0r0000gn/T/tfx-interactive-2022-01-24T13_46_15.376117-2wg_qh1h/ExampleValidator/anomalies/4/Split-eval.
INFO:absl:Running publisher for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 4
        type_id: 20
        uri: "/var/folders/fm/1byvh8pj2hbf1yl82_qv2q0r0000gn/T/tfx-interactive-2022-01-24T13_46_15.376117-2wg_qh1h/ExampleValidator/anomalies/4"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 20
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [114]:
# older version
# validation_dir = validate_stats.outputs['output'].get()[0].uri
# anomalies_path = os.path.join(validation_dir, 'anomalies.pbtxt')

# anomalies = tfdv.load_anomalies_text(anomalies_path)
# tfdv.display_anomalies(anomalies)

In [115]:
context.show(example_validator.outputs['anomalies'])

In the anomalies table, we can see that there are no anomalies. This is what we'd expect, since this the first dataset that we've analyzed and the schema is tailored to it. You should review this schema -- anything unexpected means an anomaly in the data. Once reviewed, the schema can be used to guard future data, and anomalies produced here can be used to debug model performance, understand how your data evolves over time, and identify data errors.

## Transform

https://www.tensorflow.org/tfx/guide/transform

The Transform TFX pipeline component performs feature engineering on tf.Examples emitted from an ExampleGen component, using a data schema created by a SchemaGen component, and emits both a SavedModel as well as statistics on both pre-transform and post-transform data. When executed, the SavedModel will accept tf.Examples emitted from an ExampleGen component and emit the transformed feature data.

- Consumes: tf.Examples from an ExampleGen component, and a data schema from a SchemaGen component.
- Emits: A SavedModel to a Trainer component, pre-transform and post-transform statistics.

__Configuring a Tansform component (utils.py)__ <br>
Once your preprocessing_fn is written, it needs to be defined in a python module that is then provided to the Transform component as an input. This module will be loaded by transform and the function named preprocessing_fn will be found and used by Transform to construct the preprocessing pipeline.

In [116]:
# GET UTILS.PY THAT CONFIGURES THE CNN MODEL - GROUP 2 
_module_file = os.getcwd()+'/utils.py'

In [117]:
transform = tfx.components.Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=_module_file)

context.run(transform)

INFO:absl:Generating ephemeral wheel package for '/Users/tommybillywijaya/Documents/_Main_Document/*Projects/JFC/cifar-10/utils.py' (including modules: ['utils-Copy1', 'utils', 'pipeline', 'pipeline-new']).
INFO:absl:User module package has hash fingerprint version 5718d622a418876b6ec8b996b8fb45fdb5b4242e2ec519044754b08d246773df.
INFO:absl:Executing: ['/Users/tommybillywijaya/anaconda3/bin/python', '/var/folders/fm/1byvh8pj2hbf1yl82_qv2q0r0000gn/T/tmppu44uk2a/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/var/folders/fm/1byvh8pj2hbf1yl82_qv2q0r0000gn/T/tmpk7799onp', '--dist-dir', '/var/folders/fm/1byvh8pj2hbf1yl82_qv2q0r0000gn/T/tmp6zr6bknd']
INFO:absl:Successfully built user code wheel distribution at '/var/folders/fm/1byvh8pj2hbf1yl82_qv2q0r0000gn/T/tfx-interactive-2022-01-24T13_46_15.376117-2wg_qh1h/_wheels/tfx_user_code_Transform-0.0+5718d622a418876b6ec8b996b8fb45fdb5b4242e2ec519044754b08d246773df-py3-none-any.whl'; target user module is 'utils'.
INFO:absl:Full user modu

INFO:tensorflow:Assets written to: /var/folders/fm/1byvh8pj2hbf1yl82_qv2q0r0000gn/T/tfx-interactive-2022-01-24T13_46_15.376117-2wg_qh1h/Transform/transform_graph/5/.temp_path/tftransform_tmp/63c8768a550548baa41ffaf12b40401b/assets
INFO:tensorflow:tensorflow_text is not available.
INFO:tensorflow:tensorflow_decision_forests is not available.
INFO:tensorflow:struct2tensor is not available.
INFO:tensorflow:tensorflow_text is not available.
INFO:tensorflow:tensorflow_decision_forests is not available.
INFO:tensorflow:struct2tensor is not available.


INFO:absl:Running publisher for Transform
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 5
        type_id: 22
        uri: "/var/folders/fm/1byvh8pj2hbf1yl82_qv2q0r0000gn/T/tfx-interactive-2022-01-24T13_46_15.376117-2wg_qh1h/Transform/transform_graph/5"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 22
        name: "TransformGraph"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 6
        type_id: 14
        uri: "/var/folders/fm/1byvh8pj2hbf1yl82_qv2q0r0000gn/T/tfx-interactive-2022-01-24T13_46_15.376117-2wg_qh1h/Transform/transformed_examples/5"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        base_type: DATASET
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 7
        type_id: 23
        uri: "/var/folders/fm/1byvh8pj2hbf1yl82_qv2q0r0000gn/T/tfx-interactive-2022-01-24T13_46_15.376117-2wg_qh1h/Transform/updated_analyzer_cache/5"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 23
        name: "TransformCache"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        pre_transform_schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 8
        type_id: 18
        uri: "/var/folders/fm/1byvh8pj2hbf1yl82_qv2q0r0000gn/T/tfx-interactive-2022-01-24T13_46_15.376117-2wg_qh1h/Transform/pre_transform_schema/5"
        custom_properties {
          key: "name"
          value {
            string_value: "pre_transform_schema"
          }
        }
        custom_properties {
          key: "pr

Let's examine the output artifacts of Transform. This component produces two types of outputs:

- transform_graph is the graph that can perform the preprocessing operations (this graph will be included in the serving and evaluation models).
- transformed_examples represents the preprocessed training and evaluation data.

In [118]:
transform.outputs

{'transform_graph': Channel(
     type_name: TransformGraph
     artifacts: [Artifact(artifact: id: 5
 type_id: 22
 uri: "/var/folders/fm/1byvh8pj2hbf1yl82_qv2q0r0000gn/T/tfx-interactive-2022-01-24T13_46_15.376117-2wg_qh1h/Transform/transform_graph/5"
 custom_properties {
   key: "name"
   value {
     string_value: "transform_graph"
   }
 }
 custom_properties {
   key: "producer_component"
   value {
     string_value: "Transform"
   }
 }
 custom_properties {
   key: "state"
   value {
     string_value: "published"
   }
 }
 custom_properties {
   key: "tfx_version"
   value {
     string_value: "1.5.0"
   }
 }
 state: LIVE
 , artifact_type: id: 22
 name: "TransformGraph"
 )]
     additional_properties: {}
     additional_custom_properties: {}
 ),
 'transformed_examples': Channel(
     type_name: Examples
     artifacts: [Artifact(artifact: id: 6
 type_id: 14
 uri: "/var/folders/fm/1byvh8pj2hbf1yl82_qv2q0r0000gn/T/tfx-interactive-2022-01-24T13_46_15.376117-2wg_qh1h/Transform/transform

Take a peek at the transform_graph artifact. It points to a directory containing three subdirectories.

In [119]:
train_uri = transform.outputs['transform_graph'].get()[0].uri
os.listdir(train_uri)

['transformed_metadata', 'transform_fn', 'metadata']

The transformed_metadata subdirectory contains the schema of the preprocessed data. The transform_fn subdirectory contains the actual preprocessing graph. The metadata subdirectory contains the schema of the original data.

We can also take a look at the first three transformed examples:

In [128]:
# # Get the URI of the output artifact representing the transformed examples, which is a directory
# train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

# # Get the list of files in this directory (all compressed TFRecord files)
# tfrecord_filenames = [os.path.join(train_uri, name)
#                       for name in os.listdir(train_uri)]

# # Create a `TFRecordDataset` to read these files
# dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# # Iterate over the first 3 records and decode them.
# for tfrecord in dataset.take(3):
#     serialized_example = tfrecord.numpy()
#     example = tf.train.Example()
#     example.ParseFromString(serialized_example)
#     pp.pprint(example)

After the Transform component has transformed your data into features, and the next step is to train a model.

# Trainer

https://www.tensorflow.org/tfx/guide/trainer

Trainer takes:
- tf.Examples used for training and eval.
- A user provided module file that defines the trainer logic.
- Protobuf definition of train args and eval args.
- (Optional) A data schema created by a SchemaGen pipeline component and optionally altered by the developer.
- (Optional) transform graph produced by an upstream Transform component.
- (Optional) pre-trained models used for scenarios such as warmstart.
- (Optional) hyperparameters, which will be passed to user module function. Details of the integration with Tuner can be found here https://www.tensorflow.org/tfx/guide/tuner

Trainer emits: At least one model for inference/serving (typically in SavedModelFormat) and optionally another model for eval (typically an EvalSavedModel).

In [121]:
# GET UTILS.PY THAT CONFIGURES THE CNN MODEL - GROUP 2 
_module_file = os.getcwd()+'/utils.py'

In [70]:
trainer = tfx.components.Trainer(
    module_file=_module_file,
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=tfx.proto.TrainArgs(num_steps=10000),
    eval_args=tfx.proto.EvalArgs(num_steps=5000))

context.run(trainer)

INFO:absl:Generating ephemeral wheel package for '/Users/tommybillywijaya/Documents/_Main_Document/*Projects/JFC/cifar-10/utils.py' (including modules: ['utils-Copy1', 'utils', 'pipeline', 'pipeline-new']).
INFO:absl:User module package has hash fingerprint version eec75244a1c19398362077c36574afdf602a0fdd9bbb7953aadef2b8bc89645b.
INFO:absl:Executing: ['/Users/tommybillywijaya/anaconda3/bin/python', '/var/folders/fm/1byvh8pj2hbf1yl82_qv2q0r0000gn/T/tmp_as7rfcq/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/var/folders/fm/1byvh8pj2hbf1yl82_qv2q0r0000gn/T/tmp4a7tpv6c', '--dist-dir', '/var/folders/fm/1byvh8pj2hbf1yl82_qv2q0r0000gn/T/tmpen8c_kzv']
INFO:absl:Successfully built user code wheel distribution at '/var/folders/fm/1byvh8pj2hbf1yl82_qv2q0r0000gn/T/tfx-interactive-2022-01-20T14_13_15.936374-2ic85qd9/_wheels/tfx_user_code_Trainer-0.0+eec75244a1c19398362077c36574afdf602a0fdd9bbb7953aadef2b8bc89645b-py3-none-any.whl'; target user module is 'utils'.
INFO:absl:Full user module

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


INFO:absl:Model: "model_7"
INFO:absl:_________________________________________________________________
INFO:absl: Layer (type)                Output Shape              Param #   
INFO:absl:=================================================================
INFO:absl: image_raw_xf (InputLayer)   [(None, 32, 32, 3)]       0         
INFO:absl:                                                                 
INFO:absl: conv2d_14 (Conv2D)          (None, 30, 30, 64)        1792      
INFO:absl:                                                                 
INFO:absl: conv2d_15 (Conv2D)          (None, 28, 28, 32)        18464     
INFO:absl:                                                                 
INFO:absl: flatten_7 (Flatten)         (None, 25088)             0         
INFO:absl:                                                                 
INFO:absl: dense_7 (Dense)             (None, 10)                250890    
INFO:absl:                                                   

KeyboardInterrupt: 

In [122]:
trainer = tfx.components.Trainer(
    module_file=_module_file,
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=tfx.proto.TrainArgs(num_steps=101),
    eval_args=tfx.proto.EvalArgs(num_steps=51))

context.run(trainer)

INFO:absl:Generating ephemeral wheel package for '/Users/tommybillywijaya/Documents/_Main_Document/*Projects/JFC/cifar-10/utils.py' (including modules: ['utils-Copy1', 'utils', 'pipeline', 'pipeline-new']).
INFO:absl:User module package has hash fingerprint version 5718d622a418876b6ec8b996b8fb45fdb5b4242e2ec519044754b08d246773df.
INFO:absl:Executing: ['/Users/tommybillywijaya/anaconda3/bin/python', '/var/folders/fm/1byvh8pj2hbf1yl82_qv2q0r0000gn/T/tmp6u3aht0j/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/var/folders/fm/1byvh8pj2hbf1yl82_qv2q0r0000gn/T/tmpbxdhnxhv', '--dist-dir', '/var/folders/fm/1byvh8pj2hbf1yl82_qv2q0r0000gn/T/tmptk8k3056']
INFO:absl:Successfully built user code wheel distribution at '/var/folders/fm/1byvh8pj2hbf1yl82_qv2q0r0000gn/T/tfx-interactive-2022-01-24T13_46_15.376117-2wg_qh1h/_wheels/tfx_user_code_Trainer-0.0+5718d622a418876b6ec8b996b8fb45fdb5b4242e2ec519044754b08d246773df-py3-none-any.whl'; target user module is 'utils'.
INFO:absl:Full user module

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


INFO:absl:Model: "model_14"
INFO:absl:_________________________________________________________________
INFO:absl: Layer (type)                Output Shape              Param #   
INFO:absl:=================================================================
INFO:absl: image_raw_xf (InputLayer)   [(None, 32, 32, 3)]       0         
INFO:absl:                                                                 
INFO:absl: conv2d_28 (Conv2D)          (None, 30, 30, 64)        1792      
INFO:absl:                                                                 
INFO:absl: conv2d_29 (Conv2D)          (None, 28, 28, 32)        18464     
INFO:absl:                                                                 
INFO:absl: flatten_14 (Flatten)        (None, 25088)             0         
INFO:absl:                                                                 
INFO:absl: dense_14 (Dense)            (None, 10)                250890    
INFO:absl:                                                  

101/101 [==============================] - 19s 157ms/step - loss: 2.2901 - accuracy: 0.1108 - val_loss: 2.2775 - val_accuracy: 0.1330
INFO:tensorflow:tensorflow_text is not available.
INFO:tensorflow:tensorflow_decision_forests is not available.
INFO:tensorflow:struct2tensor is not available.
INFO:tensorflow:Assets written to: /var/folders/fm/1byvh8pj2hbf1yl82_qv2q0r0000gn/T/tfx-interactive-2022-01-24T13_46_15.376117-2wg_qh1h/Trainer/model/6/Format-Serving/assets


INFO:absl:Training complete. Model written to /var/folders/fm/1byvh8pj2hbf1yl82_qv2q0r0000gn/T/tfx-interactive-2022-01-24T13_46_15.376117-2wg_qh1h/Trainer/model/6/Format-Serving. ModelRun written to /var/folders/fm/1byvh8pj2hbf1yl82_qv2q0r0000gn/T/tfx-interactive-2022-01-24T13_46_15.376117-2wg_qh1h/Trainer/model_run/6
INFO:absl:Running publisher for Trainer
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Trainer
    execution_id: 6
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 13
        type_id: 25
        uri: "/var/folders/fm/1byvh8pj2hbf1yl82_qv2q0r0000gn/T/tfx-interactive-2022-01-24T13_46_15.376117-2wg_qh1h/Trainer/model/6"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 25
        name: "Model"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 14
        type_id: 26
        uri: "/var/folders/fm/1byvh8pj2hbf1yl82_qv2q0r0000gn/T/tfx-interactive-2022-01-24T13_46_15.376117-2wg_qh1h/Trainer/model_run/6"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 26
        name: "ModelRun"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

__Analyse training with Tensorboard__

Take a peek at the trainer artifact. It points to a directory containing the model subdirectories.

In [26]:
model_artifact_dir = trainer.outputs['model'].get()[0].uri
pp.pprint(os.listdir(model_artifact_dir))
model_dir = os.path.join(model_artifact_dir, 'Format-Serving')
pp.pprint(os.listdir(model_dir))

['Format-Serving']
['keras_metadata.pb', 'variables', 'saved_model.pb', 'assets']


Optionally, we can connect TensorBoard to the Trainer to analyze our model's training curves.

In [27]:
model_run_artifact_dir = trainer.outputs['model_run'].get()[0].uri

%load_ext tensorboard
%tensorboard --logdir {model_run_artifact_dir}
#

# Evaluator

The Evaluator TFX pipeline component performs deep analysis on the training results for your models, to help you understand how your model performs on subsets of your data. The Evaluator also helps you validate your exported models, ensuring that they are "good enough" to be pushed to production. https://www.tensorflow.org/tfx/guide/evaluator 

Consumes:
- An eval split from ExampleGen
- A trained model from Trainer
- A previously blessed model (if validation to be performed)

Emits:
- Analysis results to ML Metadata
- Validation results to ML Metadata (if validation to be performed)

In [123]:
OVERALL_SLICE_SPEC = evaluator_pb2.SingleSlicingSpec()

ALL_SPECS = [
    OVERALL_SLICE_SPEC,
]

In [124]:
eval_config = tfma.EvalConfig(
    slicing_specs=[tfma.SlicingSpec()]
)

In [125]:
eval_config = tfma.EvalConfig(
    model_specs=[
        # This assumes a serving model with signature 'serving_default'. If
        # using estimator based EvalSavedModel, add signature_name: 'eval' and 
        # remove the label_key.
        tfma.ModelSpec(
            signature_name='serving_default',
            label_key='label' #not 'label'
            )
        ],
    metrics_specs=[
        tfma.MetricsSpec(
            # The metrics added here are in addition to those saved with the
            # model (assuming either a keras model or EvalSavedModel is used).
            # Any metrics added into the saved model (for example using
            # model.compile(..., metrics=[...]), etc) will be computed
            # automatically.
            # To add validation thresholds for metrics saved with the model,
            # add them keyed by metric name to the thresholds map.
            metrics=[
                tfma.MetricConfig(class_name='CategoricalAccuracy',
                  threshold=tfma.MetricThreshold(
                      value_threshold=tfma.GenericValueThreshold(
                          lower_bound={'value': 0.5}),
                      # Change threshold will be ignored if there is no
                      # baseline model resolved from MLMD (first run).
                      change_threshold=tfma.GenericChangeThreshold(
                          direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                          absolute={'value': -1e-10})))
            ]
        )
    ],
    slicing_specs=[
        # An empty slice spec means the overall slice, i.e. the whole dataset.
        tfma.SlicingSpec()
    ])

In [126]:
model_resolver = tfx.dsl.Resolver(
      strategy_class=tfx.dsl.experimental.LatestBlessedModelStrategy,
      model=tfx.dsl.Channel(type=tfx.types.standard_artifacts.Model),
      model_blessing=tfx.dsl.Channel(
          type=tfx.types.standard_artifacts.ModelBlessing)).with_id(
              'latest_blessed_model_resolver')
context.run(model_resolver)

INFO:absl:Running driver for latest_blessed_model_resolver
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running publisher for latest_blessed_model_resolver
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: latest_blessed_model_resolver
    execution_id: 7
    outputs:
        model: Channel(
            type_name: Model
            artifacts: []
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_blessing: Channel(
            type_name: ModelBlessing
            artifacts: []
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [127]:
model_analyzer = tfx.components.Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)

context.run(model_analyzer)

INFO:absl:Running driver for Evaluator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Evaluator
INFO:absl:Nonempty beam arg extra_packages already includes dependency
INFO:absl:udf_utils.get_fn {'eval_config': '{\n  "metrics_specs": [\n    {\n      "metrics": [\n        {\n          "class_name": "CategoricalAccuracy",\n          "threshold": {\n            "change_threshold": {\n              "absolute": -1e-10,\n              "direction": "HIGHER_IS_BETTER"\n            },\n            "value_threshold": {\n              "lower_bound": 0.5\n            }\n          }\n        }\n      ]\n    }\n  ],\n  "model_specs": [\n    {\n      "label_key": "label",\n      "signature_name": "serving_default"\n    }\n  ],\n  "slicing_specs": [\n    {}\n  ]\n}', 'feature_slicing_spec': None, 'fairness_indicator_thresholds': 'null', 'example_splits': 'null', 'module_file': None, 'module_path': None} 'custom_eval_shared_model'
INFO:absl:Request was made to igno


Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7fb8a0227588> and <keras.engine.input_layer.InputLayer object at 0x7fb85e461b70>).


INFO:absl:The 'example_splits' parameter is not set, using 'eval' split.
INFO:absl:Evaluating model.
INFO:absl:udf_utils.get_fn {'eval_config': '{\n  "metrics_specs": [\n    {\n      "metrics": [\n        {\n          "class_name": "CategoricalAccuracy",\n          "threshold": {\n            "change_threshold": {\n              "absolute": -1e-10,\n              "direction": "HIGHER_IS_BETTER"\n            },\n            "value_threshold": {\n              "lower_bound": 0.5\n            }\n          }\n        }\n      ]\n    }\n  ],\n  "model_specs": [\n    {\n      "label_key": "label",\n      "signature_name": "serving_default"\n    }\n  ],\n  "slicing_specs": [\n    {}\n  ]\n}', 'feature_slicing_spec': None, 'fairness_indicator_thresholds': 'null', 'example_splits': 'null', 'module_file': None, 'module_path': None} 'custom_extractors'
INFO:absl:Request was made to ignore the baseline ModelSpec and any change thresholds. This is likely because a baseline model was not provided: u


Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7fb7bf714748> and <keras.engine.input_layer.InputLayer object at 0x7fb86330e6a0>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7fb72c1d4f98> and <keras.engine.input_layer.InputLayer object at 0x7fb7c18e1b70>).


ValueError: Shapes (1000, 1) and (1000, 10) are incompatible [while running 'ExtractEvaluateAndWriteResults/ExtractAndEvaluate/EvaluateMetricsAndPlots/ComputeMetricsAndPlots()/CombineMetricsPerSlice/WindowIntoDiscarding']

In [54]:
model_analyzer = tfx.components.Evaluator(
    examples=example_gen.outputs['examples'],
    model_exports=trainer.outputs['model'],
    feature_slicing_spec=evaluator_pb2.FeatureSlicingSpec(
        specs=ALL_SPECS
    ))

context.run(model_analyzer)

TypeError: __init__() got an unexpected keyword argument 'model_exports'

In [197]:
model_analyzer = tfx.components.Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    feature_slicing_spec=evaluator_pb2.FeatureSlicingSpec(
        specs=ALL_SPECS
    ))

Now let's examine the output artifacts of Evaluator.

In [201]:
PATH_TO_RESULT = model_analyzer.outputs['output'].get()[0].uri
tfma_result = tfma.load_eval_result(PATH_TO_RESULT)
tfma.view.render_slicing_metrics(tfma_result)

KeyError: 'output'

# InfraValidator

In [20]:
model_validator = ModelValidator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['output'])

context.run(model_validator)

ExecutionResult(
    component_id: ModelValidator
    execution_id: 8
    outputs:
        blessing: Channel(
            type_name: ModelBlessingPath
            artifacts: [Artifact(type_name: ModelBlessingPath, uri: /tmp/tfx-interactive-2019-09-03T20_21_55.462505-u52yuvqc/ModelValidator/blessing/8/, split: , id: 13)]
        ))

# Pusher

In [ ]:
pusher = Pusher(
    model_export=trainer.outputs['output'],
    model_blessing=model_validator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=_serving_model_dir)))

context.run(pusher)